# Set up the environment

## Import packages

In [1]:
import os

# Introduction

## Motivation
This notebook is inpired by information theory [lectures by David McKay](https://www.youtube.com/playlist?list=PLN3p8NUNcClDu1hc2m5cVp8FOEmuF3vRy). I want to implement [Huffman coding algorithm](https://en.wikipedia.org/wiki/Huffman_coding) for data compression and use it to compress [human genome](https://en.wikipedia.org/wiki/Human_genome) sequence. Why doing it? Well, I'm greately facinated by [Arithmetic coding](https://en.wikipedia.org/wiki/Arithmetic_coding) algorithm for data compression and Huffman's algorithm would be great as a beseline to compare to

## Brief overview of related concepts

### Symbol coding problem

In symbol coding problem we have a set of symbols $S = \{s_1, s_2, \ldots, s_n\}$ with probabilities $P = \{p_1, p_2, \ldots, p_n\}$ and we want to encode them using binary strings (codewords) $C = \{c_1, c_2, \ldots, c_n\}$ such that the expected codeword length is minimized. The expected codeword length $L$ is given by:

$$
L = \sum_{i=1}^{n} p_i \cdot l(c_i), \tag{1}
$$

where $l(c_i)$ is the length of codeword $c_i$. Let's look at an example to illustrate this.

---

#### Example
Let's say we have a set of symbols $S = \{A, B, C, D\}$ with probabilities $P = \{0.5, 0.25, 0.125, 0.125\}$. We want to find the optimal codewords $C$ that minimize the expected codeword length.

a) One possible solution is to assign the following codewords:
- A: 1000
- B: 0100
- C: 0010
- D: 0001

The expected codeword length $L$ is calculated as follows:
$$
L = \sum_{i=1}^{n} p_i \cdot l(c_i) = 0.5 \cdot 4 + 0.25 \cdot 4 + 0.125 \cdot 4 + 0.125 \cdot 4 = 4
$$

b) Another possible solution is to assign the following codewords:
- A: 1
- B: 01
- C: 001
- D: 0001

The expected codeword length $L$ is calculated as follows:
$$
L = \sum_{i=1}^{n} p_i \cdot l(c_i) = 0.5 \cdot 1 + 0.25 \cdot 2 + 0.125 \cdot 3 + 0.125 \cdot 4 = 1.875
$$

c) Third possible solution is to assign the following codewords:
- A: 1
- B: 00
- C: 010
- D: 10

The expected codeword length $L$ is calculated as follows:
$$
L = \sum_{i=1}^{n} p_i \cdot l(c_i) = 0.5 \cdot 1 + 0.25 \cdot 2 + 0.125 \cdot 3 + 0.125 \cdot 4 = 1.625
$$
---

Moreover we want our code to have several useful properties:
- **Uniquely decodable**: for any string $x$ and $y$ such that $x \neq y$ codewords $C(x)$ and $C(y)$ must be different $C(x) \neq C(y)$. In plain english this means that we can always decode a string of codewords back to the original symbols without ambiguity. In this light, solution (c) in example 1 is not uniquely decodable because the both string `DC` and `ABD` are encoded as $C(DC)=C(ABD) = 10010$.
- **Minimal expected codeword length**: we want to minimize the expected codeword length $L$.

**Note:** [ASCII](https://en.wikipedia.org/wiki/ASCII) code is another interesting example of symbol coding.

### Source coding theorem

A question arises: what is the minimum expected codeword length $L$ that we can achieve? The answer is given by [Shannon's source coding theorem](https://en.wikipedia.org/wiki/Shannon%27s_source_coding_theorem) which states that the minimum expected codeword length $L$ is bounded by the entropy $H$ of the source:

$$
L \geq H, \tag{2}
$$

where the entropy $H$ is defined as:

$$
H = -\sum_{i=1}^{n} p_i \log_2 p_i, \tag{3}
$$

This means that no lossless compression scheme can achieve an expected codeword length less than the entropy of the source.

By comparing equations (1) and (2) we can see that equality $L = H$ holds when codeword length is equal to $l(c_i) = -\log_2 p_i$ for all symbols $s_i$. However, this is not always possible because codeword lengths must be integers. Therefore, we can only achieve $L$ that is close to $H$.

### Huffman Coding

[Huffman coding](https://en.wikipedia.org/wiki/Huffman_coding) is a popular algorithm used for lossless data compression. The basic idea is to assign variable-length codes to input characters, with shorter codes assigned to more frequent characters. This results in a prefix-free binary code, meaning no code is a prefix of any other, which allows for efficient encoding and decoding.

The algorithm works by building a binary tree where each leaf node represents a symbol and its probability. The tree is constructed by repeatedly merging the two nodes with the lowest probabilities until only one node remains, which becomes the root of the tree. The code for each symbol is then determined by traversing the tree from the root to the leaf, assigning a '0' for a left branch and a '1' for a right branch.

#### Example

Consider the "alphabet" from previous example, where symbol probabilities are

```
A: 0.5
B: 0.25
C: 0.125
D: 0.125
```

Then the binary tree for this alphabet would look like this:

```
        *
       / \
      /   \
     *     A(0.5)
    / \
   /   \
B(0.25) *
       / \
      /   \
  C(0.125) D(0.125)
```

The algorithm assigns codes by traversing from root to leaf: left=0, right=1. So the resulting code for each symbol would be:
"
```
A: 1
B: 01
C: 001
D: 000
```

An average code length $L$ can be calculated as follows:

$$
L = \sum_{i=1}^4 p(i) \cdot l(i) = \frac{1}{2} \cdot 1 + \frac{1}{4} \cdot 2 + \frac{1}{8} \cdot 3 + \frac{1}{8} \cdot 3  = 1.75
$$

For comparison, entropy $H$ can be calculated as follows:

$$
H = - \sum_{i=1}^4 p(i) \cdot \log_2(p(i)) = \frac{1}{2} \log_22 + \frac{1}{4} \log_24 + \frac{1}{8} \log_28 + \frac{1}{8} \log_28 = 1.75
$$

As we can see, the average code length $L$ is equal to the entropy $H$ for this particular example. This is a special case and may not hold for all probability distributions, but it illustrates the relationship between Huffman coding and entropy.


# Let's play around

## Implement Huffman coding algorithm

Huffman coding algorithm follows these steps:

1. Build a priority queue (min-heap) of nodes, where each node represents a symbol and its frequency.
2. While there is more than one node in the queue:
   a. Remove the two nodes of lowest frequency from the queue.
   b. Create a new internal node with these two nodes as children and a frequency equal to the sum of their frequencies.
   c. Insert the new node back into the queue.
3. The remaining node is the root of the Huffman tree.
4. Traverse the tree to generate the Huffman codes for each symbol.
